In [6]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

# Configurer Selenium (avec ChromeDriver)
driver_path = r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\chromedriver-win64\chromedriver-win64\chromedriver.exe"  # Remplacez par le chemin de votre ChromeDriver


## **Rotten tomato extraction with selenium**

In [7]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

def scrape_and_save_to_csv(driver_path, url, output_file):
    # Configurer le service pour le chemin du ChromeDriver
    service = Service(driver_path)

    # Ajouter des options pour Chrome si nécessaire
    options = Options()
    options.add_argument("--start-maximized")  # Démarrer le navigateur maximisé
    options.add_argument("--disable-notifications")  # Désactiver les notifications

    # Initialiser le driver avec service et options
    driver = webdriver.Chrome(service=service, options=options)

    # Ouvrir l'URL
    driver.get(url)

    # Charger toutes les données en cliquant sur le bouton "Load more"
    while True:
        try:
            # Attendre que le bouton soit cliquable
            load_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, "button[data-qa='dlp-load-more-button']"))
            )
            load_more_button.click()
            time.sleep(2)  # Pause pour permettre le chargement des données
        except Exception as e:
            print("Aucun bouton 'Load more' trouvé ou fin de la liste.")
            break

    # Extraire le contenu HTML complet après avoir chargé tous les films
    html_content = driver.page_source
    driver.quit()

    # Analyser le HTML avec BeautifulSoup
    soup = BeautifulSoup(html_content, "html.parser")
    movies = soup.find_all("a", {"data-qa": "discovery-media-list-item-caption"})

    # Liste pour stocker les résultats
    scraped_data = []

    # Extraire les informations de chaque film
    for movie in movies:
        try:
            # Nom du film
            movie_name = movie.find("span", {"data-qa": "discovery-media-list-item-title"}).text.strip()
            
            # Critiques des professionnels (criticsScore)
            critics_score = movie.find("rt-text", {"slot": "criticsScore"}).text.strip()
            
            # Critiques des spectateurs (audienceScore)
            audience_score = movie.find("rt-text", {"slot": "audienceScore"}).text.strip()
            
            # Sentiments (positif ou négatif)
            critics_sentiment = movie.find("score-icon-critics")["sentiment"]
            audience_sentiment = movie.find("score-icon-audience")["sentiment"]
            
            # Ajouter les données à la liste
            scraped_data.append({
                "Movie Name": movie_name,
                "Critics Score": critics_score,
                "Critics Sentiment": critics_sentiment,
                "Audience Score": audience_score,
                "Audience Sentiment": audience_sentiment,
            })
        except Exception as e:
            print(f"Erreur lors de l'extraction d'un film : {e}")

    # Enregistrer les données dans un fichier CSV
    with open(output_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["Movie Name", "Critics Score", "Critics Sentiment", "Audience Score", "Audience Sentiment"])
        writer.writeheader()
        writer.writerows(scraped_data)

    print(f"Données enregistrées avec succès dans {output_file}")


In [9]:
url = "https://www.rottentomatoes.com/browse/movies_at_home/audience:spilled~critics:rotten"
scrape_and_save_to_csv(driver_path, url,r"rotten1.csv")

Aucun bouton 'Load more' trouvé ou fin de la liste.
Erreur lors de l'extraction d'un film : 'NoneType' object has no attribute 'text'
Données enregistrées avec succès dans rotten1.csv


In [1]:
import pandas as pd
df1 = pd.read_csv("rotten1.csv")

In [141]:
print(len(df1))
df1.head()

285


,Movie Name,Critics Score,Critics Sentiment,Audience Score,Audience Sentiment
0,Y2K,43%,negative,52%,negative
1,Joker: Folie à Deux,31%,negative,32%,negative
2,The Dead Don't Die,54%,negative,38%,negative
3,Dear Santa,24%,negative,41%,negative
4,Nutcrackers,47%,negative,52%,negative


netflix : https://www.rottentomatoes.com/browse/movies_at_home/affiliates:netflix~audience:spilled~critics:rotten

In [14]:
url = "https://www.rottentomatoes.com/browse/movies_at_home/affiliates:netflix~audience:spilled~critics:rotten"
scrape_and_save_to_csv(driver_path, url, r"rotten2.csv")

Aucun bouton 'Load more' trouvé ou fin de la liste.
Données enregistrées avec succès dans rotten2.csv


In [2]:
import pandas as pd
df2 = pd.read_csv(r"rotten2.csv")

In [143]:
len(df2)

49

In [144]:
df2.head()

,Movie Name,Critics Score,Critics Sentiment,Audience Score,Audience Sentiment
0,The Dead Don't Die,54%,negative,38%,negative
1,Holidate,46%,negative,54%,negative
2,Miller's Girl,29%,negative,44%,negative
3,The Dark Tower,16%,negative,44%,negative
4,Trigger Warning,20%,negative,15%,negative


In [18]:
url = "https://www.rottentomatoes.com/browse/movies_at_home/affiliates:paramount-plus~audience:spilled~critics:rotten"
scrape_and_save_to_csv(driver_path, url, r"rotten3.csv")

Aucun bouton 'Load more' trouvé ou fin de la liste.
Données enregistrées avec succès dans rotten3.csv


In [3]:
df3 = pd.read_csv(r"rotten3.csv")
print(len(df3))
df3.head()

15


,Movie Name,Critics Score,Critics Sentiment,Audience Score,Audience Sentiment
0,Transformers: Revenge of the Fallen,19%,negative,57%,negative
1,Nacho Libre,39%,negative,55%,negative
2,Baywatch,18%,negative,56%,negative
3,Failure to Launch,23%,negative,52%,negative
4,Scary Movie 3,35%,negative,54%,negative


In [ ]:
url = "https://www.rottentomatoes.com/browse/movies_in_theaters/sort:critic_lowest"
scrape_and_save_to_csv(driver_path, url, r"rotten4.csv")

In [4]:
df4 = pd.read_csv(r"rotten4.csv")
print(len(df4))
df4.head()

30


,Movie Name,Critics Score,Critics Sentiment,Audience Score,Audience Sentiment
0,Kraven the Hunter,17%,negative,73%,positive
1,Red One,31%,negative,90%,positive
2,Homestead,33%,negative,73%,positive
3,The Lord of the Rings: The War of the Rohirrim,45%,negative,83%,positive
4,Mufasa: The Lion King,56%,negative,88%,positive


In [ ]:
url = "https://www.rottentomatoes.com/browse/movies_in_theaters/sort:popular"
scrape_and_save_to_csv(driver_path, url, r"rotten5.csv")

In [9]:
df5 = pd.read_csv(r"rotten5.csv")
print(len(df5))
df5.head()

30


,Movie Name,Critics Score,Critics Sentiment,Audience Score,Audience Sentiment
0,Nosferatu,86%,positive,76%,positive
1,Mufasa: The Lion King,56%,negative,88%,positive
2,Sonic the Hedgehog 3,88%,positive,97%,positive
3,A Complete Unknown,79%,positive,96%,positive
4,Red One,31%,negative,90%,positive


In [ ]:
url = "https://www.rottentomatoes.com/browse/movies_at_home/sort:a_z"
scrape_and_save_to_csv(driver_path, url, r"rotten6.csv")

In [11]:
df6 = pd.read_csv(r"rotten6.csv")
print(len(df6))
df6.head()

49


,Movie Name,Critics Score,Critics Sentiment,Audience Score,Audience Sentiment
0,"""Sr.""",97%,positive,90%,positive
1,#AMFAD: All My Friends Are Dead,50%,negative,54%,negative
2,#Alive,88%,positive,63%,positive
3,#Horror,46%,negative,10%,negative
4,#LIKE,75%,positive,59%,negative


In [12]:
df_combined = pd.concat([df1, df2, df3,df4,df5,df6], ignore_index=True)

In [13]:
len(df_combined)

458

In [14]:
# Supprimer les doublons en se basant sur le nom du film
df_combined.drop_duplicates(subset=["Movie Name"], keep="first", inplace=True)

In [15]:
len(df_combined)

407

In [16]:
df_combined.to_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\rottentomatoes_movies.csv", index=False)

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Fonction d'extraction des informations d'un film à partir de Rotten Tomatoes
def get_movie_info(movie_name):
    # Nettoyage du nom du film pour l'utiliser dans l'URL (remplacer les espaces par des underscores)
    movie_url_name = movie_name.replace(" ", "_").lower()
    url = f"https://www.rottentomatoes.com/m/{movie_url_name}"

    # Effectuer une requête pour récupérer la page
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")

    # Extraction du synopsis
    synopsis = soup.find("drawer-more", {"maxlines": "2"})
    synopsis_text = synopsis.find("rt-text", {"slot": "content"}).text.strip() if synopsis else "N/A"

    # Extraction de l'année, de la durée et du genre
    metadata = soup.find_all("rt-text", {"slot": "metadataProp"})
    release_date = metadata[0].text.strip() if metadata else "N/A"  # Date de sortie
    duration = metadata[1].text.strip() if len(metadata) > 1 else "N/A"  # Durée
    genres = [genre.text.strip() for genre in soup.find_all("rt-text", {"slot": "metadataGenre"})]  # Genres

    # Extraction des critiques
    reviews = []
    review_section = soup.find("section", {"aria-labelledby": "critics-reviews-label"})
    if review_section:
        review_cards = review_section.find_all("media-review-card-critic")
        for card in review_cards:
            critic_name = card.find("rt-link", {"slot": "displayName"}).text.strip()
            review_text = card.find("rt-text", {"slot": "content"}).text.strip()
            reviews.append((critic_name, review_text))

    # Retourner les résultats sous forme de dictionnaire
    return {
        "movie_name": movie_name,
        "synopsis": synopsis_text,
        "release_date": release_date,
        "duration": duration,
        "genres": genres,
        "reviews": reviews
    }


movies_info = df_combined["Movie Name"].apply(get_movie_info)

# Transformer les informations extraites en un DataFrame
df_movie_details = pd.json_normalize(movies_info)

# Affichage des informations extraites
print(df_movie_details)


                   movie_name  \
0                         Y2K   
1         Joker: Folie à Deux   
2          The Dead Don't Die   
3                  Dear Santa   
4                 Nutcrackers   
..                        ...   
402                      1945   
403                      1985   
404  1987: When the Day Comes   
405                      1992   
406                       1BR   

                                              synopsis           release_date  \
0    A CIA agent (Ed O'Ross), a veteran (Louis Goss...                      R   
1                                                  N/A                    N/A   
2                                                  N/A                    N/A   
3    Gordon Covington (D.L. Green) doesn't get to s...                      G   
4    A strait-laced man finds his life suddenly upe...                   2024   
..                                                 ...                    ...   
402  A village is forced to face up 

In [18]:
df_movie_details.head()

,movie_name,synopsis,release_date,duration,genres,reviews
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",R,1999,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc..."
1,Joker: Folie à Deux,N/A,N/A,N/A,[],[]
2,The Dead Don't Die,N/A,N/A,N/A,[],[]
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,G,1998,"[Holiday, Comedy, Fantasy]",[]
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,1h 44m,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar..."


In [19]:
df_movie_details.to_csv(r"rotten_details")

In [20]:
na_threshold = 4
def count_na(row):
    return sum([1 for value in row if value == "N/A" or (isinstance(value, list) and len(value) == 0)])

# Appliquer cette fonction à chaque ligne de votre DataFrame
df_movie_details["na_count"] = df_movie_details.apply(count_na, axis=1)

# Filtrer les films qui ont un nombre de N/A inférieur ou égal au seuil
df_filtered_movies = df_movie_details[df_movie_details["na_count"] <= na_threshold]

# Supprimer la colonne "na_count" qui n'est plus nécessaire
df_filtered_movies = df_filtered_movies.drop(columns=["na_count"])

In [21]:
df_filtered_movies.head()

,movie_name,synopsis,release_date,duration,genres,reviews
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",R,1999,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc..."
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,G,1998,"[Holiday, Comedy, Fantasy]",[]
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,1h 44m,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar..."
5,Our Little Secret,Two resentful exes are forced to spend Christm...,Released Nov 27,1h 40m,"[Holiday, Romance, Comedy]","[(Brooks Eisenbise, Under the shiny Netflix-fe..."
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,PG,"Released Nov 16, 1996","[Kids & Family, Holiday, Comedy]","[(Chris Stuckmann, That rare family-themed Chr..."


In [22]:
len(df_filtered_movies)

307

In [23]:
def fix_released_in_duration(df):
    for idx, row in df.iterrows():
        # Si 'Released' est dans la colonne 'duration', déplacer dans 'release_date' et mettre la durée correcte
        if isinstance(row["duration"], str) and "Released" in row["duration"]:
            # Déplacer la valeur 'Released' vers release_date
            df.at[idx, "release_date"] = row["duration"]
            # Mettre la durée dans 'duration', ici on suppose que la durée reste dans la même cellule
            df.at[idx, "duration"] = None  # Ou placez une valeur par défaut si vous le souhaitez, comme 'Non spécifié'
    
    return df

In [24]:
df_corrected = fix_released_in_duration(df_filtered_movies)
df_corrected.head()

,movie_name,synopsis,release_date,duration,genres,reviews
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",R,1999,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc..."
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,G,1998,"[Holiday, Comedy, Fantasy]",[]
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,1h 44m,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar..."
5,Our Little Secret,Two resentful exes are forced to spend Christm...,Released Nov 27,1h 40m,"[Holiday, Romance, Comedy]","[(Brooks Eisenbise, Under the shiny Netflix-fe..."
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,"Released Nov 16, 1996",None,"[Kids & Family, Holiday, Comedy]","[(Chris Stuckmann, That rare family-themed Chr..."


In [25]:
import re
def extract_year_from_released(df):
    current_year = 2024  # Valeur par défaut si l'année n'est pas trouvée
    
    for idx, row in df.iterrows():
        release_value = row["release_date"]
        
        if isinstance(release_value, str) and "Released" in release_value:
            # Extraction de l'année à l'aide d'une expression régulière
            match = re.search(r'(\d{4})', release_value)  # Chercher 4 chiffres consécutifs
            if match:
                year = match.group(1)
            else:
                year = current_year  # Si l'année n'est pas trouvée, utiliser 2024
            
            # Mettre à jour la colonne release_date avec l'année
            df.at[idx, "release_date"] = year
    
    return df

# Appliquer la fonction
df_corrected = extract_year_from_released(df_corrected)
df_corrected.head()

,movie_name,synopsis,release_date,duration,genres,reviews
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",R,1999,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc..."
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,G,1998,"[Holiday, Comedy, Fantasy]",[]
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,1h 44m,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar..."
5,Our Little Secret,Two resentful exes are forced to spend Christm...,2024,1h 40m,"[Holiday, Romance, Comedy]","[(Brooks Eisenbise, Under the shiny Netflix-fe..."
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,1996,None,"[Kids & Family, Holiday, Comedy]","[(Chris Stuckmann, That rare family-themed Chr..."


In [26]:
def move_invalid_duration_to_release(df):
    for idx, row in df.iterrows():
        duration_value = row["duration"]
        
        # Vérifier si la durée contient "h" ou "min" (si c'est une durée valide)
        if isinstance(duration_value, str) and not ('h' in duration_value or 'min' in duration_value):
            # Si la durée n'est pas valide, déplacer la valeur vers release_date
            df.at[idx, "release_date"] = duration_value
            df.at[idx, "duration"] = None  # Effacer la valeur de duration
            
    return df


df_corrected = move_invalid_duration_to_release(df_corrected)

In [27]:
df_corrected.head()

,movie_name,synopsis,release_date,duration,genres,reviews
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",1999,None,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc..."
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,1998,None,"[Holiday, Comedy, Fantasy]",[]
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,1h 44m,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar..."
5,Our Little Secret,Two resentful exes are forced to spend Christm...,2024,1h 40m,"[Holiday, Romance, Comedy]","[(Brooks Eisenbise, Under the shiny Netflix-fe..."
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,1996,None,"[Kids & Family, Holiday, Comedy]","[(Chris Stuckmann, That rare family-themed Chr..."


In [28]:
print(df_corrected["duration"].value_counts(dropna=False))


duration
None      208
1h 30m      9
1h 40m      6
1h 20m      5
1h 25m      5
1h 32m      4
1h 31m      4
1h 28m      4
1h 33m      4
1h 44m      4
1h 35m      3
1h 26m      3
1h 43m      3
1h 42m      3
1h 55m      3
2h 0m       3
1h 21m      3
1h 27m      3
1h 45m      2
1h 34m      2
1h 24m      2
1h 22m      2
2h 14m      1
1h 15m      1
1h 49m      1
1h 10m      1
1h 46m      1
1h 38m      1
1h 37m      1
1h 23m      1
1h 13m      1
1h 17m      1
2h 7m       1
2h 6m       1
2h 35m      1
2h 11m      1
1h 47m      1
3h 12m      1
1h 16m      1
2h 8m       1
1h 5m       1
3h 3m       1
1h 58m      1
1h 54m      1
Name: count, dtype: int64


In [29]:
import pandas as pd
import re

# Fonction pour convertir les durées en minutes
def convert_to_minutes(duration):
    if isinstance(duration, str):  # Vérifie si c'est une chaîne de caractères
        hours = 0
        minutes = 0
        
        # Recherche des heures (ex: 2h)
        hours_match = re.search(r'(\d+)\s*h', duration)
        if hours_match:
            hours = int(hours_match.group(1))
        
        # Recherche des minutes (ex: 30m)
        minutes_match = re.search(r'(\d+)\s*m', duration)
        if minutes_match:
            minutes = int(minutes_match.group(1))
        
        # Conversion en minutes
        total_minutes = (hours * 60) + minutes
        return total_minutes
    return None  # Retourne None si la durée n'est pas une chaîne valide

# Appliquer la fonction à la colonne 'duration'
df_corrected['duration_min'] = df_corrected['duration'].apply(convert_to_minutes)

df_corrected.head()


,movie_name,synopsis,release_date,duration,genres,reviews,duration_min
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",1999,None,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc...",NaN
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,1998,None,"[Holiday, Comedy, Fantasy]",[],NaN
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,1h 44m,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar...",104.0
5,Our Little Secret,Two resentful exes are forced to spend Christm...,2024,1h 40m,"[Holiday, Romance, Comedy]","[(Brooks Eisenbise, Under the shiny Netflix-fe...",100.0
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,1996,None,"[Kids & Family, Holiday, Comedy]","[(Chris Stuckmann, That rare family-themed Chr...",NaN


In [30]:
df_corrected.drop(columns=['duration'])

,movie_name,synopsis,release_date,genres,reviews,duration_min
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",1999,"[Mystery & Thriller, Action]","[(Jorge Avila Andrade, Adems de que la producc...",NaN
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,1998,"[Holiday, Comedy, Fantasy]",[],NaN
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,"[Holiday, Comedy, Drama]","[(Jesse Hassenger, What Green adds to familiar...",104.0
5,Our Little Secret,Two resentful exes are forced to spend Christm...,2024,"[Holiday, Romance, Comedy]","[(Brooks Eisenbise, Under the shiny Netflix-fe...",100.0
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,1996,"[Kids & Family, Holiday, Comedy]","[(Chris Stuckmann, That rare family-themed Chr...",NaN
...,...,...,...,...,...,...
399,18 to Party,"Teens grapple with a spate of recent suicides,...",2020,[Comedy],"[(Tim Cogshell, I loved this angsty little per...",80.0
402,1945,A village is forced to face up to its ill-gott...,2017,[Drama],"[(Leslie Felperin, [A] sombre, accomplished bu...",91.0
403,1985,When an adventurous teen discovers a secret ma...,2016,"[Comedy, Drama, Adventure]",[],114.0
405,1992,Martin is 17 and spends all day recording ever...,30m,"[Comedy, Drama]",[],NaN


In [31]:
df_corrected.to_csv(r"rotten_final.csv")

In [32]:
# Transformer les listes de la colonne 'genres' en chaînes de caractères séparées par ";"
df_corrected['genres'] = df_corrected['genres'].apply(lambda x: "; ".join(x) if isinstance(x, list) else x)


In [33]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time


# Ajouter des colonnes pour le cast et les producteurs
df_corrected["producer"] = ""
df_corrected["cast"] = ""

# Étape 2 : Fonction pour extraire les informations du site
def get_movie_details(movie_name):
    # Remplacer les espaces par des underscores pour construire l'URL
    movie_url_name = movie_name.replace(" ", "_").lower()
    base_url = f"https://www.rottentomatoes.com/m/{movie_url_name}"
    cast_url = f"{base_url}#cast-and-crew"
    
    try:
        # Requête pour récupérer les données
        response = requests.get(base_url)
        response.raise_for_status()  # Vérifie les erreurs HTTP
        soup = BeautifulSoup(response.text, "html.parser")
        
        # Extraire les producteurs
        producers = []
        producer_section = soup.find_all("div", {"slot": "content"})
        if producer_section:
            producers = [p.text.strip() for p in producer_section if "Producer" in p.text]
        
        # Extraire le cast
        cast = []
        cast_section = soup.find_all("a", {"data-qa": "person-item"})
        for person in cast_section:
            name = person.find("p", {"data-qa": "person-name"}).text.strip()
            role = person.find("p", {"data-qa": "person-role"}).text.strip()
            cast.append(f"{name} ({role})")
        
        return ", ".join(producers), ", ".join(cast)
    
    except Exception as e:
        print(f"Erreur lors de l'extraction des données pour {movie_name}: {e}")
        return "", ""

# Étape 3 : Boucle sur chaque film du dataset
for index, row in df_corrected.iterrows():
    movie_name = row["movie_name"]  # Remplace par le nom de ta colonne pour les titres des films
    print(f"Extraction pour le film : {movie_name}...")
    producer, cast = get_movie_details(movie_name)
    df_corrected.at[index, "producer"] = producer
    df_corrected.at[index, "cast"] = cast
    time.sleep(2)  # Pause pour éviter d'être bloqué par le site

# Étape 4 : Sauvegarder le fichier mis à jour
output_file = "rotten_final.csv"
df_corrected.to_csv(output_file, index=False, encoding="utf-8")
print(f"Fichier mis à jour sauvegardé : {output_file}")


Extraction pour le film : Y2K...
Extraction pour le film : Dear Santa...
Extraction pour le film : Nutcrackers...
Extraction pour le film : Our Little Secret...
Extraction pour le film : Jingle All the Way...
Extraction pour le film : Megalopolis...
Extraction pour le film : Four Christmases...
Extraction pour le film : Here...
Extraction pour le film : Madame Web...
Extraction pour le film : Noelle...
Extraction pour le film : Surviving Christmas...
Extraction pour le film : The Merry Gentlemen...
Extraction pour le film : Fifty Shades of Grey...
Extraction pour le film : The Inheritance...
Extraction pour le film : Borderlands...
Extraction pour le film : Office Christmas Party...
Extraction pour le film : Fred Claus...
Extraction pour le film : A Bad Moms Christmas...
Extraction pour le film : Candy Cane Lane...
Extraction pour le film : Absolution...
Extraction pour le film : Home Sweet Home Alone...
Extraction pour le film : Love the Coopers...
Extraction pour le film : The Mounta

In [34]:
df_corrected["producer"] = df_corrected["producer"].str.replace(r'[\n\r]+', '', regex=True)


In [35]:
output_file = "rotten_final.csv"
df_corrected.to_csv(output_file)

In [36]:
df_corrected.drop(columns=['duration'])

,movie_name,synopsis,release_date,genres,reviews,duration_min,producer,cast
0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",1999,Mystery & Thriller; Action,"[(Jorge Avila Andrade, Adems de que la producc...",NaN,,"Richard Pepin (Director), Louis Gossett Jr. (M..."
3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,1998,Holiday; Comedy; Fantasy,[],NaN,,"Fred Olen Ray (Director), D.L. Green (Gordon),..."
4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,Holiday; Comedy; Drama,"[(Jesse Hassenger, What Green adds to familiar...",104.0,,"David Gordon Green (Director), Ben Stiller (Mi..."
5,Our Little Secret,Two resentful exes are forced to spend Christm...,2024,Holiday; Romance; Comedy,"[(Brooks Eisenbise, Under the shiny Netflix-fe...",100.0,,"Stephen Herek (Director), Lindsay Lohan (Avery..."
6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,1996,Kids & Family; Holiday; Comedy,"[(Chris Stuckmann, That rare family-themed Chr...",NaN,,"Brian Levant (Director), Arnold Schwarzenegger..."
...,...,...,...,...,...,...,...,...
399,18 to Party,"Teens grapple with a spate of recent suicides,...",2020,Comedy,"[(Tim Cogshell, I loved this angsty little per...",80.0,,"Jeff Roda (Director), Alivia Clark (Amy), Tann..."
402,1945,A village is forced to face up to its ill-gott...,2017,Drama,"[(Leslie Felperin, [A] sombre, accomplished bu...",91.0,,"Ferenc Török (Director), Péter Rudolf (Szentes..."
403,1985,When an adventurous teen discovers a secret ma...,2016,Comedy; Drama; Adventure,[],114.0,,"Kang Vang (Director), Chang Yang (Billy aka Be..."
405,1992,Martin is 17 and spends all day recording ever...,30m,Comedy; Drama,[],NaN,,"Anthony Doncque (Director), Mathieu Dessertine..."


In [37]:
df_corrected.columns = df_corrected.columns.str.upper()

Calcul du rating 

In [38]:
df1 = pd.read_csv('rotten_final.csv')  
df2 = pd.read_csv('rottentomatoes_movies.csv') 


In [39]:
df2.columns = df2.columns.str.upper()

In [40]:
df2.columns

Index(['MOVIE NAME', 'CRITICS SCORE', 'CRITICS SENTIMENT', 'AUDIENCE SCORE',
       'AUDIENCE SENTIMENT'],
      dtype='object')

In [41]:
df1.columns

Index(['Unnamed: 0', 'movie_name', 'synopsis', 'release_date', 'duration',
       'genres', 'reviews', 'duration_min', 'producer', 'cast'],
      dtype='object')

In [42]:
df1.columns = df1.columns.str.upper()

In [43]:
df2.rename(columns={'MOVIE NAME': 'MOVIE_NAME'}, inplace=True)

In [44]:
# Fusionner les deux DataFrames sur la colonne 'Movie Name'
merged_df = pd.merge(df1, df2, on='MOVIE_NAME', how='inner')

merged_df['CRITICS SCORE'] = merged_df['CRITICS SCORE'].str.replace('%', '').astype(int)
merged_df['AUDIENCE SCORE'] = merged_df['AUDIENCE SCORE'].str.replace('%', '').astype(int)
merged_df['CRITICS SCORE'] = merged_df['CRITICS SCORE'] / 100
merged_df['AUDIENCE SCORE'] = merged_df['AUDIENCE SCORE'] / 100

# Calculer le rating sur 5
merged_df['RATING_SUR_5'] = ((merged_df['CRITICS SCORE'] + merged_df['AUDIENCE SCORE']) / 2) * 5


In [45]:
merged_df.drop(columns=['DURATION','CRITICS SCORE','AUDIENCE SCORE','AUDIENCE SENTIMENT','CRITICS SENTIMENT'])

,UNNAMED: 0,MOVIE_NAME,SYNOPSIS,RELEASE_DATE,GENRES,REVIEWS,DURATION_MIN,PRODUCER,CAST,RATING_SUR_5
0,0,Y2K,"A CIA agent (Ed O'Ross), a veteran (Louis Goss...",1999,Mystery & Thriller; Action,"[('Jorge Avila Andrade', 'Adems de que la prod...",NaN,NaN,"Richard Pepin (Director), Louis Gossett Jr. (M...",2.375
1,3,Dear Santa,Gordon Covington (D.L. Green) doesn't get to s...,1998,Holiday; Comedy; Fantasy,[],NaN,NaN,"Fred Olen Ray (Director), D.L. Green (Gordon),...",1.625
2,4,Nutcrackers,A strait-laced man finds his life suddenly upe...,2024,Holiday; Comedy; Drama,"[('Jesse Hassenger', 'What Green adds to famil...",104.0,NaN,"David Gordon Green (Director), Ben Stiller (Mi...",2.475
3,5,Our Little Secret,Two resentful exes are forced to spend Christm...,2024,Holiday; Romance; Comedy,"[('Brooks Eisenbise', 'Under the shiny Netflix...",100.0,NaN,"Stephen Herek (Director), Lindsay Lohan (Avery...",2.225
4,6,Jingle All the Way,Workaholic Howard Langston (Arnold Schwarzeneg...,1996,Kids & Family; Holiday; Comedy,"[('Chris Stuckmann', ""That rare family-themed ...",NaN,NaN,"Brian Levant (Director), Arnold Schwarzenegger...",1.525
...,...,...,...,...,...,...,...,...,...,...
302,399,18 to Party,"Teens grapple with a spate of recent suicides,...",2020,Comedy,"[('Tim Cogshell', 'I loved this angsty little ...",80.0,NaN,"Jeff Roda (Director), Alivia Clark (Amy), Tann...",3.925
303,402,1945,A village is forced to face up to its ill-gott...,2017,Drama,"[('Leslie Felperin', '[A] sombre, accomplished...",91.0,NaN,"Ferenc Török (Director), Péter Rudolf (Szentes...",4.500
304,403,1985,When an adventurous teen discovers a secret ma...,2016,Comedy; Drama; Adventure,[],114.0,NaN,"Kang Vang (Director), Chang Yang (Billy aka Be...",4.450
305,405,1992,Martin is 17 and spends all day recording ever...,30m,Comedy; Drama,[],NaN,NaN,"Anthony Doncque (Director), Mathieu Dessertine...",3.800


In [47]:

df = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\rotten_final.csv")
df.to_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\rotten_final.csv", index=False)

In [ ]:
df1 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\rotten_final.csv")
df2 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\last_Target.csv")

In [3]:
import pandas as pd

# Charger les deux fichiers CSV
df1 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\rotten-files\rotten_final.csv")
df2 = pd.read_csv(r"C:\Users\Aycha\Desktop\M2_BDIA\NLP\Projet_movie\final\last_Target.csv")
df1.columns = df1.columns.str.upper()
df2.columns = df2.columns.str.upper()
rename_dict = {
    'MOVIE_NAME': 'MOVIE_NAME',
    'SYNOPSIS': 'SYNOPSIS',
    'RELEASE_DATE': 'YEAR',
    'GENRES': 'GENRES',
    'REVIEWS': 'REVIEWS',
    'DURATION_MIN': 'DURATION_MIN',
    'PRODUCER': 'PRODUCERS',
    'CAST': 'CAST',
    'RATING_SUR_5': 'RATING_SUR_5'
}

df1.rename(columns=rename_dict, inplace=True)  # Renommer les colonnes
# Concaténer les DataFrames (par défaut, concatène verticalement)



In [4]:
df1.columns

Index(['UNNAMED: 0', 'MOVIE_NAME', 'SYNOPSIS', 'YEAR', 'DURATION', 'GENRES',
       'REVIEWS', 'DURATION_MIN', 'PRODUCERS', 'CAST'],
      dtype='object')

In [5]:
df2.columns

Index(['UNNAMED: 0', 'MOVIE_NAME', 'REVIEWS', 'SYNOPSIS', 'YEAR', 'CAST',
       'DURATION_MIN', 'GENRES', 'PRODUCERS', 'RATING_SUR_5', 'MISSING_COUNT'],
      dtype='object')

In [6]:
df2.drop(columns=['MISSING_COUNT'], inplace=True)

In [7]:
df_combined = pd.concat([df2, df1], ignore_index=True)
df_combined.to_csv('rotten_letter', index=False)

In [8]:
df_combined.columns

Index(['UNNAMED: 0', 'MOVIE_NAME', 'REVIEWS', 'SYNOPSIS', 'YEAR', 'CAST',
       'DURATION_MIN', 'GENRES', 'PRODUCERS', 'RATING_SUR_5', 'DURATION'],
      dtype='object')

In [9]:
df_combined.drop(columns=['UNNAMED: 0','DURATION'], inplace=True)

In [10]:
df_combined.to_csv('rotten_letter.csv', index=False)

In [11]:
df_combined

,MOVIE_NAME,REVIEWS,SYNOPSIS,YEAR,CAST,DURATION_MIN,GENRES,PRODUCERS,RATING_SUR_5
0,carry-on,Aucune critique disponible,A cul-de-sac in an oppressive suburb becomes a...,1998.0,"Julia Stiles, William R. Moses, Patrick Muldoo...",88.0,Thriller,Frank Beddor; Greg Steinberg,2.64
1,nosferatu,I only watched this to understand the Spongebo...,Years after witnessing the death of the revere...,2024.0,"Paul Mescal, Denzel Washington, Pedro Pascal, ...",148.0,Drama; Action; Adventure,Michael A. Pruss; Winston Azzopardi; David Fra...,3.37
2,home-alone,Young Patrick Bateman**do you think the wet ba...,After receiving an unexpected call from her wa...,2024.0,"Auliʻi Cravalho, Dwayne Johnson, Hualālai Chun...",100.0,Comedy; Animation; Adventure; Family,Christina Chen; Yvett Merino Flores,2.87
3,the-holdovers,There's a handshake in this that feels more ov...,A fading celebrity decides to use a black mark...,2024.0,"Demi Moore, Margaret Qualley, Dennis Quaid, Ed...",141.0,Horror; Science Fiction,Coralie Fargeat; Eric Fellner; Tim Bevan,3.85
4,anora,A terrifying tale of dating a mama’s boy**Sean...,After discovering their significant others are...,2024.0,"Lindsay Lohan, Ian Harding, Kristin Chenoweth,...",99.0,Drama; Romance; Comedy,Mike Elliott; Lisa Gooding,2.36
...,...,...,...,...,...,...,...,...,...
3530,18 to Party,"[('Tim Cogshell', 'I loved this angsty little ...","Teens grapple with a spate of recent suicides,...",2020,"Jeff Roda (Director), Alivia Clark (Amy), Tann...",80.0,Comedy,NaN,NaN
3531,1945,"[('Leslie Felperin', '[A] sombre, accomplished...",A village is forced to face up to its ill-gott...,2017,"Ferenc Török (Director), Péter Rudolf (Szentes...",91.0,Drama,NaN,NaN
3532,1985,[],When an adventurous teen discovers a secret ma...,2016,"Kang Vang (Director), Chang Yang (Billy aka Be...",114.0,Comedy; Drama; Adventure,NaN,NaN
3533,1992,[],Martin is 17 and spends all day recording ever...,30m,"Anthony Doncque (Director), Mathieu Dessertine...",NaN,Comedy; Drama,NaN,NaN


In [12]:
len(df_combined)

3535